In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as sm

In [2]:
data = pd.read_csv('stata_data.csv')
data.head()

,Unnamed: 0,CI,NASDAQ,NSE,NYSE,OTCMKTS,TGT,TPE,WMT,one_m_diff,three_m_diff,six_m_diff,one_y_diff
0,0,0,0,0,1,0,0,0,0,-674.048964,180.833666,120.936221,33.318400
1,1,1,1,0,0,0,0,0,0,-184.660191,183.459660,188.346252,-59.873427
2,2,0,1,0,0,0,0,0,0,-104.197427,-146.805878,-137.526202,-28.649221
3,3,1,0,0,0,0,0,1,0,-217.692308,-180.464697,-138.247863,-76.223776
4,4,1,0,0,1,0,0,0,0,117.842129,632.339710,-433.098300,171.927897


In [3]:
data.describe()

,Unnamed: 0,CI,NASDAQ,NSE,NYSE,OTCMKTS,TGT,TPE,WMT,one_m_diff,three_m_diff,six_m_diff,one_y_diff
count,72.00000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,64.000000,64.000000,64.000000,64.000000
mean,35.50000,0.500000,0.194444,0.013889,0.597222,0.069444,0.013889,0.013889,0.013889,-inf,-103.821945,-204.238043,-133.515541
std,20.92845,0.503509,0.398550,0.117851,0.493899,0.255992,0.117851,0.117851,0.117851,NaN,698.485820,618.448478,861.555750
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,-4498.516621,-3295.530440,-5254.652160
25%,17.75000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-244.239925,-162.055827,-236.439955,-106.620789
50%,35.50000,0.500000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-100.177646,-64.898496,-85.126340,-54.409284
75%,53.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-18.262311,19.696866,-21.492702,20.342816
max,71.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1736.059346,2609.007883,1125.549457,1521.637427


In [6]:
onemresult = sm.ols(formula="one_m_diff ~ NYSE + CI", data=data).fit()
print(onemresult.summary)

<bound method RegressionResults.summary of <statsmodels.regression.linear_model.OLSResults object at 0x7fc214fba3d0>>


In [ ]:
threemresult = sm.ols(formula="three_m_diff ~ NYSE + CI", data=data).fit()
threemresult.summary

In [ ]:
sixmresult = sm.ols(formula="six_m_diff ~ NYSE + CI", data=data).fit()
sixmresult.summary

In [ ]:
oneyresult = sm.ols(formula="one_y_diff ~ NYSE + CI", data=data).fit()
oneyresult.summary